In [16]:
import pandas as pd
import numpy as np
import string
import nltk #biblioteca pentru cuvinte
from nltk.corpus import stopwords
nltk.download('stopwords')
english_words = stopwords.words('english')

from sklearn.model_selection import train_test_split


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Fabi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
data = pd.read_csv("enronSpamSubset.csv")
data = data.fillna("empty")

train_data, test_data = train_test_split(data, test_size=0.2)
train_data = train_data.fillna("empty")
test_data = test_data.fillna("empty")

In [40]:
from nltk.tokenize import wordpunct_tokenize
cuvinte_curate = wordpunct_tokenize(data.iloc[1]['Body'])
print(cuvinte_curate)
cuvinte_curate = [cuv.lower() for cuv in cuvinte_curate]
print("_______")
print(cuvinte_curate)
print("_______")
cuvinte_curate = ['[num]' if cuv.isdigit() else cuv for cuv in cuvinte_curate]
print(cuvinte_curate)


def curatare_text(text_body):
    # am putea sa le punem in functii separate
    cuvinte_curate = wordpunct_tokenize(data.iloc[1]['Body'])
    cuvinte_curate = [cuv.lower() for cuv in cuvinte_curate]
    cuvinte_curate = ['[num]' if cuv.isdigit() else cuv for cuv in cuvinte_curate]
    cuvinte_curate = [cuv for cuv in cuvinte_curate if cuv not in english_words]
    cuvinte_curate = [cuv for cuv in cuvinte_curate if cuv not in string.punctuation]
    return cuvinte_curate


print("_______")
print(curatare_text(data.iloc[1]['Body']))

['Subject', ':', 'are', 'you', 'listed', 'in', 'major', 'search', 'engines', '?', 'submitting', 'your', 'website', 'in', 'search', 'engines', 'may', 'increase', 'your', 'online', 'sales', 'dramatically', '.', 'lf', 'you', 'invested', 'time', 'and', 'money', 'into', 'your', 'website', ',', 'you', 'simply', 'must', 'submit', 'your', 'website', 'oniine', 'otherwise', 'it', 'wili', 'be', 'invisible', 'virtuaiiy', ',', 'which', 'means', 'efforts', 'spent', 'in', 'vain', '.', 'lf', 'you', 'want', 'peopie', 'to', 'know', 'about', 'your', 'website', 'and', 'boost', 'your', 'revenues', ',', 'the', 'only', 'way', 'to', 'do', 'that', 'is', 'to', 'make', 'your', 'site', 'visible', 'in', 'places', 'where', 'people', 'search', 'for', 'information', ',', 'i', '.', 'e', '.', 'submit', 'your', 'website', 'in', 'multipie', 'search', 'engines', '.', 'submit', 'your', 'website', 'online', 'and', 'watch', 'visitors', 'stream', 'to', 'your', 'e', '-', 'business', '.', 'best', 'reqards', ',', 'brittnyatkins'

In [66]:
from tqdm import tqdm
from collections import Counter

class CLASIFICATORUL():
    def __init__(self, train_data):
        self.data = train_data
        self.label_map = {
            0: "NON-SPAM",
            1: "SPAM",
        }
        self.apriori_probs = {
            1: 0.05,
            0: 0.95,
        }
        self.spam_words = []
        self.ham_words = []
        
        self.prob_maps = [
            {} for i in range(len(self.label_map))
        ]
    
    def clean_text(self, body):
        cuvinte_curate = wordpunct_tokenize(body)
        cuvinte_curate = [cuv.lower() for cuv in cuvinte_curate]
        cuvinte_curate = ['[num]' if cuv.isdigit() else cuv for cuv in cuvinte_curate]
        cuvinte_curate = [cuv for cuv in cuvinte_curate if cuv not in english_words]
        cuvinte_curate = [cuv for cuv in cuvinte_curate if cuv not in string.punctuation]
        return cuvinte_curate
        
    
    
    def _prepare_data(self):
        self.spam_words = []
        self.ham_words = []
        print("PREPARING DATA....")
        for key, row in tqdm(self.data.iterrows()):
            body = row['Body']
            label = row['Label']
            
            words = self.clean_text(body)
            if label == 1:
                self.spam_words += words
            else:
                self.ham_words += words
        
        print("DONE..")
        
    def build_prob_map(self, word_list, label):
        
        self.prob_maps[label] = {}
        
        c = Counter(word_list)
        total_words_no = sum(c.values())
        
        for w, w_count in c.items():
            self.prob_maps[label][w] = (w_count+1)/(total_words_no + 2)
            
    def compute_log_prob(self, words, label):
        
        log_prob = 0
        unknown_words = 0
        
        for w in words:
            if w in self.prob_maps[label]:
                log_prob += np.log(self.prob_maps[label][w])
            else:
                log_prob += np.log(1e-7)
                unknown_words += 1
                
        log_prob += np.log(self.apriori_probs[label])

        return log_prob, unknown_words
            
            
            
    def fit(self):
        self.build_prob_map(self.ham_words, 0)
        self.build_prob_map(self.spam_words, 1)
    
    def inference(self, text):
        words = self.clean_text(text)
        unknown = []
        result = []
        for i in [0,1]:
            log_prob, unknown_words = self.compute_log_prob(words, i)
            result.append(log_prob)
            unknown.append(unknown_words)
            
        return result, unknown
    
    def predict(self, text):
        logs, unknown = self.inference(text)
        return np.argmax(logs), unknown
        

In [64]:
spam_classifier = CLASIFICATORUL(train_data)
spam_classifier._prepare_data()
spam_classifier.fit()

PREPARING DATA....


8000it [00:20, 392.16it/s]


DONE..


In [70]:
spam_classifier.predict("I am a nigger prince. Please giff credit card so I can send you much money!! Please send details. Offer!")

(1, [4, 3])

In [72]:
def evaluate_model(model, testing_data):
    
    from sklearn.metrics import  classification_report
    
    good = 0
    labels = []
    predictions = []
    unknowns = []
    for key, row in tqdm(testing_data.iterrows()):
        body = row['Body']
        label = row['Label']
        labels.append(label)
        
        prediction, unknown = spam_classifier.predict(body)
        predictions.append(prediction)
        unknowns.append(unknown)
    
    print(classification_report(labels, predictions))
    
    return unknowns

In [73]:
unknowns = evaluate_model(spam_classifier, test_data)

2000it [00:07, 252.72it/s]

              precision    recall  f1-score   support

           0       0.96      0.99      0.98       988
           1       0.99      0.96      0.98      1012

    accuracy                           0.98      2000
   macro avg       0.98      0.98      0.98      2000
weighted avg       0.98      0.98      0.98      2000

